In [183]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import os
import json

In [184]:
log_data = []
log_directory = './processes_logs'
for filename in os.listdir(log_directory):
    if filename.endswith('.json'):  # Проверяем, что файл имеет расширение .json
        file_path = os.path.join(log_directory, filename)  # Полный путь к файлу
        try:
            with open(file_path, 'r') as file:
                data = json.load(file)  # Загружаем данные из JSON-файла
                # Проверяем структуру JSON
                if (
                    isinstance(data, dict) and 
                    'is_working_mode' in data and 
                    'timestamp' in data and 
                    'processes' in data and 
                    isinstance(data['processes'], list)
                ):
                    log_data.append(data)  # Добавляем данные в список
        except (json.JSONDecodeError, IOError) as e:
            print(f"Ошибка при обработке файла {filename}: {e}")

In [185]:
# Подготовка данных
processes = []
labels = []

for entry in log_data:
    processes.append(' '.join(entry["processes"]))
    entry_is_work_mode = entry["is_working_mode"]
    labels.append(entry_is_work_mode)

In [186]:
# Кодируем метки (True/False) в числа (1/0)
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [187]:
# Преобразование строк в числовые векторы
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(processes)
sequences = tokenizer.texts_to_sequences(processes)
X = tf.keras.preprocessing.sequence.pad_sequences(sequences)

In [188]:
# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [189]:
# Нормализация входных данных X
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
# X = X_scaled

In [190]:
model = None
try:
    model = load_model('./predict_processes.h5')
except: 
    print("Saved model not found")
if not model:
    # Создание модели
    model = Sequential()
    model.add(Dense(64, activation='sigmoid', input_dim=X_train.shape[1]))  # Входной слой
    model.add(Dense(32, activation='sigmoid'))  # Скрытый слой
    model.add(Dense(1, activation='sigmoid'))  # Выходной слой
    # Компиляция модели
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Обучение модели
    model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=1)
    model.save('./predict_processes.h5')

In [191]:
# Оценка модели
y_test = np.where(y_test == 1, 0, 1)
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

[0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1]
[1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0]
1/1 [==============================] - 0s 100ms/step - loss: 0.8628 - accuracy: 0.6500
Accuracy: 65.00%


In [192]:
# Прогнозирование
def predict_is_work_mode(procs):
    seq = tokenizer.texts_to_sequences([procs])
    padded = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=X_train.shape[1])
    prediction = model.predict(padded)
    return prediction[0][0] > 0.5  # И возвращаем True/False

In [193]:
# Пример использования
# test_processes = ["proc1 proc2"]
# print(f'Is Work Mode: {predict_is_work_mode(test_processes)}')